In [ ]:
import os
import torch
import gc
import re
from datasets import load_dataset
from dotenv import load_dotenv
from huggingface_hub import login
os.chdir("../")
from src.schemas.finetuning import FinetuningData
from src.configs import (
    SFTTrainerConfig, ModelConfig, TokenizerConfig,
    LoRAConfig, TrainingArgs, BnBConfig, DatasetConfig, Push2HubConfig
)
from src.tuning.trainer import FineTuner

In [ ]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
# print(torch.cuda.memory_allocated())

In [ ]:
def flush():
  gc.collect()
  torch.cuda.empty_cache()
  torch.cuda.reset_peak_memory_stats()

In [ ]:
llama_dataset = load_dataset("mlabonne/guanaco-llama2-1k", split="train")

In [ ]:
def gemma_prompt_format(example: str) -> str:
    example["sentence1"] = 'My sentence: ' + example["sentence1"]
    return example

re.findall(re.escape('<s>')+"(.*)"+re.escape("</s>"), "<s>tgygcvytgivrzy</s>")[0]


In [ ]:
torch.cuda.max_memory_reserved()

In [ ]:
data = FinetuningData(
    llmodel_config=ModelConfig(pretrained_model_name_or_path="mistralai/Mistral-7B-Instruct-v0.1", device_map="auto"),
    bnb_config=BnBConfig(),
    dataset_config=DatasetConfig(name="konkinit/guanaco-llama2-100"),
    lora_config=LoRAConfig(),
    sfttrainer_config=SFTTrainerConfig(),
    tokenizer_config=TokenizerConfig(),
    training_args=TrainingArgs(),
    push2hub_config=Push2HubConfig(finetuned_model_name="Mistral-7B-Instruct-v0.1_finetuned", commit_message="Llama-2-7b-chat-hf finetuned")
)

In [ ]:
finetuner = FineTuner(data)

In [ ]:
flush()

In [ ]:
finetuner._train()

In [ ]:
torch.cuda.max_memory_allocated() / 1024 / 1024 / 1024

In [ ]:
finetuner._push_to_HF_hub()